# Análise de Evasão de Clientes - Telecom X

## 📖 Contexto

Neste projeto, nosso objetivo é realizar uma análise exploratória detalhada sobre os dados de uma empresa de telecomunicações fictícia, a Telecom X. O foco principal é identificar os fatores que mais contribuem para a evasão de clientes, conhecida como **Churn**. Compreender os motivos que levam os clientes a cancelar seus serviços é crucial para desenvolver estratégias de retenção eficazes e melhorar a satisfação geral.

## 📌 Extração
A primeira etapa do nosso processo de análise consiste na extração dos dados. Nesta fase, carregamos as bibliotecas necessárias para manipulação e visualização de dados e importamos as informações do arquivo `TelecomX_Data.json` para o nosso ambiente de trabalho. A extração correta é fundamental para garantir que temos a matéria-prima necessária para as próximas etapas de transformação e análise.

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette('Set2')

with open('TelecomX_Data.json', 'r') as file:
    data = json.load(file)

## 🔧 Transformação
Com os dados extraídos, a próxima etapa é a **transformação**. O arquivo JSON original possui uma estrutura aninhada, o que dificulta a análise direta. Para resolver isso, criamos uma função para achatar os dados, transformando-os em um formato tabular (linhas e colunas) e armazenando-os em um DataFrame do Pandas.

Além disso, realizamos as seguintes transformações e limpezas:
- Conversão das colunas `TotalCharges` e `MonthlyCharges` para o tipo numérico, tratando valores não numéricos.
- Ajuste da coluna `SeniorCitizen` para o tipo inteiro.
- Padronização dos valores da coluna `Churn`, substituindo valores vazios por 'No'.
- Criação de uma coluna binária `Churn_Binary` (1 para 'Yes', 0 para 'No') para facilitar cálculos e análises futuras.

In [2]:
def flatten_customer_data(customer_record):
    flattened = {}
    
    flattened['customerID'] = customer_record.get('customerID', '')
    flattened['Churn'] = customer_record.get('Churn', '')
    
    customer_info = customer_record.get('customer', {})
    for key, value in customer_info.items():
        flattened[key] = value
    
    phone_info = customer_record.get('phone', {})
    for key, value in phone_info.items():
        flattened[key] = value
    
    internet_info = customer_record.get('internet', {})
    for key, value in internet_info.items():
        flattened[key] = value
    
    account_info = customer_record.get('account', {})
    for key, value in account_info.items():
        if key == 'Charges':
            charges = value
            flattened['MonthlyCharges'] = charges.get('Monthly', 0)
            flattened['TotalCharges'] = charges.get('Total', '0')
        else:
            flattened[key] = value
    
    return flattened

flattened_data = [flatten_customer_data(record) for record in data]
df = pd.DataFrame(flattened_data)

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['MonthlyCharges'] = pd.to_numeric(df['MonthlyCharges'], errors='coerce')
df['SeniorCitizen'] = df['SeniorCitizen'].astype(int)

df['Churn'] = df['Churn'].replace('', 'No')
df['Churn_Binary'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Lidando com valores ausentes
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

print(f'Shape do DataFrame: {df.shape}')
print(f'\nColunas disponíveis:')
print(df.columns.tolist())
print(f'\nPrimeiras linhas:')
df.head()

Shape do DataFrame: (7267, 22)

Colunas disponíveis:
['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn_Binary']

Primeiras linhas:


...

## 📊 Carga e Análise Exploratória (EDA)
Nesta seção, iniciamos a **Análise Exploratória de Dados (EDA)**. O objetivo é obter uma compreensão inicial do conjunto de dados, verificando sua estrutura, tipos de dados, estatísticas descritivas e a presença de valores ausentes. Esta é uma etapa essencial para guiar nossas análises futuras e a construção de visualizações relevantes.

In [3]:
print('=== ANÁLISE EXPLORATÓRIA DE DADOS ===')
print(f'\nInformações gerais do dataset:')
df.info()

=== ANÁLISE EXPLORATÓRIA DE DADOS ===

Informações gerais do dataset:
<class 'pandas.core.frame.DataFrame'>
...

In [4]:
print(f'\nEstatísticas descritivas:')
df.describe()

### Análise da Variável Alvo: Churn
O primeiro passo da nossa análise é entender a distribuição da nossa variável alvo, o `Churn`. Os gráficos abaixo mostram a proporção de clientes que evadiram em comparação com os que permaneceram.

**Insights:**
- A taxa de churn geral é de aproximadamente **25.72%**. Isso significa que cerca de um em cada quatro clientes cancelou seu serviço.
- A maioria dos clientes (**74.28%**) permaneceu na base, mas a taxa de evasão é significativa e justifica uma investigação mais aprofundada para identificar suas causas.

In [5]:
print(f"\nDistribuição da Evasão (Churn):\n{df['Churn'].value_counts()}\n")
churn_rate = (df['Churn'].value_counts(normalize=True)['Yes'] * 100)
print(f'Taxa de Evasão (Churn): {churn_rate:.2f}%\n')

fig = px.pie(df, names='Churn', title='Distribuição de Evasão (Churn) - Interativo', hole=0.3)
fig.show()

### Análise Demográfica vs. Churn
A seguir, analisamos a relação entre as características demográficas dos clientes (gênero, se são idosos, se possuem parceiro(a) e se possuem dependentes) e a taxa de churn.

**Insights:**
- **Gênero**: A taxa de churn é muito semelhante entre homens e mulheres, sugerindo que o gênero não é um fator determinante para a evasão.
- **Idosos (SeniorCitizen)**: Clientes idosos apresentam uma taxa de churn consideravelmente maior (**40.3%**) em comparação com clientes não idosos (**22.9%**). Isso pode indicar que os produtos ou serviços não estão atendendo bem a este grupo ou que eles são mais sensíveis a preço.
- **Parceiro(a) (Partner)**: Clientes sem parceiro(a) têm uma taxa de evasão maior (**32.0%**) do que os que possuem (**19.0%**). Relacionamentos podem ser um fator de estabilidade.
- **Dependentes (Dependents)**: De forma semelhante, clientes sem dependentes tendem a evadir mais (**30.3%**) do que aqueles que os possuem (**14.9%**). A presença de dependentes parece ser um forte fator de retenção.

In [6]:
demographic_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Análise Demográfica vs. Churn', fontsize=20)
axes = axes.ravel()

for i, col in enumerate(demographic_features):
    churn_by_col = df.groupby(col)['Churn'].value_counts(normalize=True).unstack()
    churn_by_col.plot(kind='pie', y='Yes', ax=axes[i], autopct='%1.1f%%', legend=False, 
                        title=f'Churn por {col}', textprops={'fontsize': 12})
    axes[i].set_ylabel('')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

### Análise de Faturamento e Tempo de Contrato (Tenure) vs. Churn
Analisamos agora as variáveis contínuas: `tenure` (tempo em meses que o cliente permanece na empresa), `MonthlyCharges` (cobranças mensais) e `TotalCharges` (cobranças totais). Os histogramas mostram a distribuição desses valores para clientes que evadiram (Churn=Yes) e os que não evadiram (Churn=No).

**Insights:**
- **Permanência (Tenure)**: O gráfico de `tenure` mostra claramente que clientes com **pouco tempo de contrato** (baixa permanência) têm uma taxa de churn muito mais alta. A evasão é significativamente maior nos primeiros meses de serviço, diminuindo drasticamente à medida que o cliente permanece mais tempo na empresa.
- **Cobranças Mensais (MonthlyCharges)**: Clientes com **cobranças mensais mais altas** tendem a evadir mais. Há uma concentração de churn na faixa de R$70 a R$100, indicando que este pode ser um ponto de preço sensível.
- **Cobranças Totais (TotalCharges)**: Clientes com **baixas cobranças totais** têm maior probabilidade de churn. Isso está diretamente relacionado ao `tenure`, já que clientes novos (baixo tenure) naturalmente terão um total de cobranças menor.

In [7]:
fig_tenure = px.histogram(df, x='tenure', color='Churn', title='Distribuição de Permanência (Tenure) por Status de Evasão (Churn)', barmode='overlay', nbins=30)
fig_tenure.show()

fig_monthly = px.histogram(df, x='MonthlyCharges', color='Churn', title='Distribuição de Cobranças Mensais por Status de Evasão (Churn)', barmode='overlay', nbins=30)
fig_monthly.show()

fig_total = px.histogram(df, x='TotalCharges', color='Churn', title='Distribuição de Cobranças Totais por Status de Evasão (Churn)', barmode='overlay', nbins=30)
fig_total.show()

### Análise de Serviços e Contrato vs. Taxa de Churn
Por fim, investigamos como diferentes tipos de serviços e contratos se relacionam com a taxa de churn. Estes gráficos de barras mostram o percentual de evasão para cada categoria das variáveis selecionadas.

**Insights:**
- **Serviço de Internet (InternetService)**: Clientes com serviço de **Fibra Óptica** têm a maior taxa de churn (**40.6%**), quase o dobro da taxa de clientes com DSL (**18.4%**). Isso pode sinalizar problemas de qualidade, instabilidade ou uma percepção de custo-benefício ruim para o serviço de fibra.
- **Contrato (Contract)**: Este é um dos fatores mais impactantes. Clientes com contrato **Mês a Mês (Month-to-month)** apresentam uma taxa de churn altíssima de **41.3%**. Em contraste, contratos de **Um Ano (10.9%)** e **Dois Anos (2.8%)** têm taxas de evasão drasticamente menores, evidenciando que contratos de longo prazo são uma poderosa ferramenta de retenção.
- **Forma de Pagamento (PaymentMethod)**: Clientes que utilizam **cheque eletrônico (Electronic check)** como forma de pagamento têm a taxa de churn mais elevada (**43.8%**). Clientes com pagamentos automáticos (transferência bancária ou cartão de crédito) apresentam taxas muito menores, sugerindo que a conveniência e a automação do pagamento contribuem para a retenção.

In [8]:
categorical_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'InternetService', 'Contract', 'PaymentMethod']

fig = make_subplots(rows=4, cols=2, subplot_titles=categorical_features, vertical_spacing=0.1)

for i, col in enumerate(categorical_features):
    row = i // 2 + 1
    col_pos = i % 2 + 1
    churn_rate_by_cat = df.groupby(col)['Churn_Binary'].mean().reset_index()
    churn_rate_by_cat['Churn_Rate_%'] = churn_rate_by_cat['Churn_Binary'] * 100
    
    fig.add_trace(go.Bar(x=churn_rate_by_cat[col], y=churn_rate_by_cat['Churn_Rate_%'], name=col), row=row, col=col_pos)

fig.update_layout(height=1200, title_text="Taxa de Churn (%) por Variável Categórica", showlegend=False)
fig.show()

# 📜 Relatório Final da Análise de Evasão da Telecom X

## 1. Introdução

Este relatório apresenta uma análise detalhada sobre a evasão de clientes (Churn) na empresa Telecom X. O principal objetivo foi identificar os perfis de clientes com maior probabilidade de cancelamento e os fatores que mais influenciam essa decisão. A análise foi realizada com base no arquivo de dados `TelecomX_Data.json` e as conclusões aqui apresentadas podem servir como base para a criação de estratégias focadas na retenção de clientes.

## 2. Passos da Análise

O processo de análise foi dividido nas seguintes etapas:

#### a. Extração e Transformação dos Dados
Os dados foram extraídos de um arquivo JSON com estrutura aninhada. Foi necessário aplicar um processo de transformação para "achatar" os dados em um formato tabular, adequado para análise. Durante esta fase, também foram realizadas as seguintes limpezas e preparações:
- Conversão de colunas de cobrança para formato numérico.
- Tratamento de 11 registros com valores ausentes na coluna `TotalCharges`, substituindo-os pela mediana da coluna para não impactar a distribuição.
- Padronização e criação da variável alvo `Churn_Binary`.

#### b. Análise Exploratória e Visualização
Com os dados limpos e estruturados, iniciamos a análise exploratória para entender as características dos clientes e como elas se relacionam com o churn. A taxa geral de churn identificada foi de **25.72%**.

As principais visualizações criadas para extrair insights foram:
- **Gráficos de Pizza e Barras**: Para analisar a taxa de churn em relação a variáveis demográficas e de serviços.
- **Histogramas**: Para observar a distribuição de variáveis contínuas (tempo de contrato, cobrança mensal e total) entre clientes que evadiram e os que permaneceram.

## 3. Insights e Conclusões

A análise revelou padrões claros e fatores de risco associados à evasão de clientes:

**Perfil do Cliente com Alto Risco de Churn:**
- **Clientes Novos**: A taxa de churn é altíssima nos primeiros meses. Clientes com baixo `tenure` são os mais propensos a cancelar.
- **Clientes com Contrato Mês a Mês**: Este tipo de contrato apresenta uma taxa de evasão de **41.3%**, sendo o principal indicador de risco.
- **Clientes com Internet de Fibra Óptica**: Surpreendentemente, este serviço possui uma taxa de churn de **40.6%**, sugerindo possíveis problemas de qualidade, preço ou concorrência.
- **Clientes com Cobranças Mensais Elevadas**: A evasão aumenta significativamente para clientes na faixa de R$70-R$100 mensais.
- **Forma de Pagamento por Cheque Eletrônico**: Esta forma de pagamento está associada a uma taxa de churn de **43.8%**, muito superior a métodos automáticos.
- **Clientes Idosos e Sem Dependentes**: Idosos e clientes sem parceiro(a) ou dependentes também mostraram uma maior propensão à evasão.

**Fatores de Retenção (Baixo Risco de Churn):**
- **Contratos de Longo Prazo**: Contratos de 1 ou 2 anos são extremamente eficazes na retenção, com taxas de churn de apenas **10.9%** e **2.8%**, respectivamente.
- **Clientes Antigos**: Quanto maior o `tenure`, menor a chance de churn.
- **Pagamentos Automáticos**: Clientes que utilizam débito automático ou cartão de crédito têm taxas de evasão muito menores.
- **Clientes com Dependentes**: A presença de dependentes é um forte indicador de retenção (**14.9%** de churn).

## 4. Recomendações e Próximos Passos

Com base nos insights gerados, recomendamos as seguintes ações:

1.  **Estratégias para Contratos de Longo Prazo**: Criar campanhas e oferecer incentivos (descontos, benefícios exclusivos) para que clientes com contrato mês a mês migrem para planos anuais ou bianuais.
2.  **Investigar o Serviço de Fibra Óptica**: Realizar uma análise de causa-raiz para entender por que a taxa de churn é tão alta neste serviço. Pode envolver pesquisas de satisfação, análise técnica da infraestrutura e comparação de preços com concorrentes.
3.  **Atenção aos Clientes Novos**: Implementar um programa de *onboarding* e acompanhamento próximo nos primeiros meses para garantir que o cliente perceba o valor do serviço e tenha uma boa experiência inicial.
4.  **Incentivar Pagamentos Automáticos**: Oferecer pequenos descontos ou benefícios para clientes que aderirem ao pagamento automático, reduzindo o atrito no processo de pagamento e o risco de cancelamento por inadimplência ou esquecimento.
5.  **Ações de Retenção Preditiva**: Como próximo passo, recomendamos o desenvolvimento de um **modelo de machine learning** para prever a probabilidade de churn de cada cliente individualmente. Isso permitiria à Telecom X agir de forma proativa, direcionando ofertas de retenção personalizadas para os clientes com maior risco de evasão antes que eles decidam cancelar.